LLM控制面板

In [1]:
from model import *
from IPython.display import display

model_list = {
    "gpt2": "../my_models/gpt2",
    "llama_13b": "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-13b",
    "llama-2-7b-chat": "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-2-7b-chat-hugging",
    "llama-2-13b-chat": "/home/cs/yangyuchen/guoyiqiu/my_models/Llama-2-13b-chat-ms",
    "vicuna_7b": "/home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/",
    "internlm-chat-7b": "/home/cs/yangyuchen/yushengliao/Medical_LLM/internlm-chat-7b",
    "internlm-chat-20b": "/home/cs/yangyuchen/guoyiqiu/my_models/internlm-chat-20b",
    "vicuna-33b-v1.3": "/home/cs/yangyuchen/guoyiqiu/my_models/models--lmsys--vicuna-33b-v1.3/snapshots/ef8d6becf883fb3ce52e3706885f761819477ab4",
}

panel = LLMPanel(model_list, chat_template=ChatTemplate.INTERNLM_TEMPLATE)
display(panel)

/mnt/petrelfs/guoyiqiu/miniconda3/envs/med_llm_gyq/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


No GPU device on this machine


LLMPanel(children=(HBox(children=(Dropdown(description='Model:', options=(('gpt2', '../my_models/gpt2'), ('lla…

Using pad_token, but it is not set yet.


Everything is ready. Time cost: 2.02s


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
import torch


input_ids1 = torch.tensor([100,200,300]).unsqueeze(0)
gen_out1 = panel.mt.model.generate(input_ids=input_ids1, max_new_tokens=5)
print('gen_out1: ', gen_out1)

input_ids2 = torch.tensor([200,200,300]).unsqueeze(0)
gen_out2 = panel.mt.model.generate(input_ids=input_ids2, max_new_tokens=5)
print('gen_out2: ', gen_out2)

source_id = 100
target_id = 200
target_embedding = panel.mt.embedding(torch.tensor(target_id).unsqueeze(0))
print('target_embedding: ', target_embedding.shape)

emb_called_count = 0

def edit_embedding(module, input, output):
    global emb_called_count
    print("embedding is called!")
    emb_called_count += 1
    input = input[0]
    print(f"original_input:{input} {input.shape}")
    print(f"original_output:{output.shape}")
    source_id_idx = input == source_id
    output[source_id_idx,:] = target_embedding
    print(f"modified_output:{output.shape}")
    return output

with LLMHooker(panel.mt, LLMHookerConfig('embedding', retain_output=False, edit_output=edit_embedding)) as hooker:
    try:
        gen_out3 = panel.mt.model.generate(input_ids=input_ids1, max_new_tokens=5)
        print('gen_out3: ', gen_out3)
    finally:
        print('emb_called_count: ', emb_called_count)
        


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


gen_out1:  tensor([[100, 200, 300,  84,  12,  75,  84,  12]])
gen_out2:  tensor([[ 200,  200,  300,   84, 1087,   68,  388,  796]])
target_embedding:  torch.Size([1, 768])
embedding is called!
original_input:tensor([[100, 200, 300]]) torch.Size([1, 3])
original_output:torch.Size([1, 3, 768])
modified_output:torch.Size([1, 3, 768])
embedding is called!
original_input:tensor([[84]]) torch.Size([1, 1])
original_output:torch.Size([1, 1, 768])
modified_output:torch.Size([1, 1, 768])
embedding is called!
original_input:tensor([[1087]]) torch.Size([1, 1])
original_output:torch.Size([1, 1, 768])
modified_output:torch.Size([1, 1, 768])
embedding is called!
original_input:tensor([[68]]) torch.Size([1, 1])
original_output:torch.Size([1, 1, 768])
modified_output:torch.Size([1, 1, 768])
embedding is called!
original_input:tensor([[388]]) torch.Size([1, 1])
original_output:torch.Size([1, 1, 768])
modified_output:torch.Size([1, 1, 768])
gen_out3:  tensor([[ 100,  200,  300,   84, 1087,   68,  388,  7

In [36]:
list(torch.tensor([1,2,3]).unsqueeze(0).squeeze().numpy())

[1, 2, 3]

Honesty Detection

In [ ]:
import os
import torch
from model import *
from sklearn.decomposition import PCA
import numpy as np
    

def calcu_score(neuron_act, reading_vectors, mean_vector, std_vector):
    """
    reading_vectors: [layer, hidden_size]
    mean_vector: [layer * hidden_size]
    std_vector: [layer * hidden_size]
    """
    neuron_act = neuron_act.transpose(0,1) # [seq_len, layer, hidden_size]
    seq_len, layer, hidden_size = neuron_act.shape
    neuron_act = neuron_act.reshape(seq_len, layer * hidden_size) # [seq_len, layer * hidden_size]
    neuron_act = (neuron_act - mean_vector) / std_vector
    neuron_act = neuron_act.reshape(seq_len, layer, hidden_size) # [seq_len, layer, hidden_size]
    scores = (neuron_act * reading_vectors.repeat(seq_len, 1, 1)).sum(-1) # [seq_len, layer]
    return scores.mean()


def compute_reading_vectors(neuron_acts, sign=False):
    '''
    neuron_acts : List[Tuple[torch.Tensor [layer, window_size, hidden_size]]]
    '''
    diff = []
    for (act1,act2) in neuron_acts:
        l = min(act1.shape[1],act2.shape[1])
        diff.append(act1[:,:l,:] - act2[:,:l,:])
    diff = torch.cat(diff, dim=1).transpose(0,1) # [sum(seq_len), layer, hidden_size]
    sample_size, n_layer, hidden_size = diff.shape
    diff = diff.reshape(sample_size, -1) # [sample_size, layer * hidden_size]
    mean_vector = torch.mean(diff, dim=0) # [layer * hidden_size]
    std_vector = torch.std(diff, dim=0) # [layer * hidden_size]
    diff = (diff.numpy() - mean_vector.numpy()) / std_vector.numpy()
    pca = PCA(n_components=1)
    pca.fit(diff)
    reading_vectors = pca.components_[0].reshape(n_layer, hidden_size) # [layer, hidden_size]
    # print('reading_vectors: ', reading_vectors.shape)
    reading_vectors = torch.from_numpy(reading_vectors) # [layer, hidden_size]
    if sign:
        acts_t = torch.cat([act1 for (act1,act2) in neuron_acts], dim=1).transpose(0,1) # [sum(seq_len), layer, hidden_size]
        acts_f = torch.cat([act2 for (act1,act2) in neuron_acts], dim=1).transpose(0,1)
        acts_t = ((acts_t.reshape(acts_t.shape[0], -1) - mean_vector) / std_vector).reshape(acts_t.shape)
        acts_f = ((acts_f.reshape(acts_f.shape[0], -1) - mean_vector) / std_vector).reshape(acts_f.shape)
        t_score = (acts_t * reading_vectors.repeat(acts_t.shape[0], 1, 1)).mean()
        f_score = (acts_f * reading_vectors.repeat(acts_f.shape[0], 1, 1)).mean()
        if t_score < f_score:
            reading_vectors = -reading_vectors
    return reading_vectors, mean_vector, std_vector, pca.explained_variance_ratio_[0]


def collect_neuron_acts(mt, dst, capture_window=(0,0), layers=None, local_bsz=32):
    data_bsz = local_bsz // 2
    dst = [dst[i:i+data_bsz] for i in range(0, len(dst), data_bsz)]
    neuron_acts = []
    for batch_pairs in tqdm(dst):
        pairs = []
        for pair in batch_pairs:
            pairs += pair
        prompt_lens = [len(mt.tok(s['input'])['input_ids']) for s in pairs]
        seq_lens = [len(mt.tok(s['input']+s['output'])['input_ids']) for s in pairs]
        with PaddingSide(mt.tok, 'right'):
            input_ids = mt.tok([s['input']+s['output'] for s in pairs], return_tensors='pt', padding=True)['input_ids']
        hook_configs = [LLMHookerConfig(module_name='block', layer=l) for l in layers]
        with torch.no_grad(), LLMHooker(mt, hook_configs) as hooker:
            mt.model(input_ids=input_ids.to(mt.model.device))
            sentences_repr = torch.stack([h.outputs[0] for h in hooker.hooks]).transpose(0,1) # [bsz, layer, seq_len, hidden_size]
        batch_neuron_acts = []
        for i,repr in enumerate(sentences_repr):
            prompt_len = prompt_lens[i]
            seq_len = seq_lens[i]
            start = prompt_len + capture_window[0] if capture_window[0] >= 0 else seq_len + capture_window[0]
            end = prompt_len + capture_window[1] if capture_window[1] > 0 else seq_len + capture_window[1]
            batch_neuron_acts.append(repr[:,start:end,:])
        batch_neuron_acts = [[batch_neuron_acts[i],batch_neuron_acts[i+1]] for i in range(0, len(batch_neuron_acts), 2)]
        neuron_acts.extend(batch_neuron_acts)
    return neuron_acts # [layer, window_size, hidden_size]


def evaluate_detection(mt, train_dst, test_dst, capture_window, compare_window, layers, local_bsz=64):
    neuron_acts = collect_neuron_acts(mt, train_dst, capture_window, layers, local_bsz=local_bsz)
    rv, mv, sv, importance = compute_reading_vectors(neuron_acts, sign=True)
    test_neuron_acts = collect_neuron_acts(mt, test_dst, compare_window, layers, local_bsz=local_bsz)
    scores = [[calcu_score(tna, rv, mv, sv),calcu_score(fna, rv, mv, sv)] for (tna, fna) in test_neuron_acts]
    mean_diff = np.mean([s[0]-s[1] for s in scores])
    acc = sum([1 for s in scores if s[0]>s[1]])/len(scores)
    return acc, mean_diff, importance


In [ ]:
import wandb
import json
import random

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
os.environ["WANDB_NOTEBOOK_NAME"] = "re.ipynb"

mt = LLM.from_pretrained(model_path="/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-2-7b-chat-hugging").cuda(get_free_gpus()[0])

tf_dst = json.load(open("data/true_false_dataset.json"))

# IID_Hard
reserved_topic = "Medical"
train_dst_iid_hard = [[td,fd] for (td,fd) in tf_dst if td['topic'] != "Medical"]
test_dst_iid_hard = [[td,fd] for (td,fd) in tf_dst if td['topic'] == "Medical"]

# IID_Weak
random.shuffle(tf_dst)
train_dst_iid_weak = tf_dst[:len(train_dst_iid_hard)]
test_dst_iid_weak = tf_dst[len(train_dst_iid_hard):]

# OOD
reserved_topic = "Medical"
prompt = "USER:Tell me a fact.\nAssistant:"
train_dst_ood = [[td,fd] for (td,fd) in tf_dst if td['topic'] != "Medical"]
test_dst_ood = [[dict(input=prompt,output=td['output'],topic=td['topic'],label=True),
                 dict(input=prompt,output=fd['output'],topic=fd['topic'],label=False)] 
                for (td,fd) in tf_dst if td['topic'] == "Medical"]

# Layer Sweep
def layer_sweep():
    # Layer Sweep
    config = {
        "capture_window": (0,0),
        "compare_window": (0,0),
        "local_bsz": 32,
    }

    wandb.init(config=config, 
            project="lat layer sweep", 
            name="vicuna_33b_seed42",
            dir="output/lat_layer_sweep",
            job_type="inference")

    for l in range(mt.n_layer):
        layers = [l]
        acc, mean_diff, importance = evaluate_detection(mt=mt, train_dst=train_dst_iid_weak, test_dst=test_dst_iid_weak,layers=layers, **config)
        print(f"iid weak\nimportance: {importance:.4f}\nacc: {acc:.4f},\nmean_diff: {mean_diff:.4f}")
        wandb.log({"iid weak": acc if acc>0.5 else 1-acc, "mean_diff": abs(mean_diff), "importance": importance}, step=l)
        acc, mean_diff, importance = evaluate_detection(mt=mt, train_dst=train_dst_iid_hard, test_dst=test_dst_iid_hard,layers=layers, **config)
        print(f"iid hard\nimportance: {importance:.4f}\nacc: {acc:.4f},\nmean_diff: {mean_diff:.4f}")
        wandb.log({"iid hard": acc if acc>0.5 else 1-acc, "mean_diff": abs(mean_diff), "importance": importance}, step=l)
        acc, mean_diff, importance = evaluate_detection(mt=mt, train_dst=train_dst_ood, test_dst=test_dst_ood, layers=layers, **config)
        print(f"ood\nimportance: {importance:.4f}\nacc: {acc:.4f},\nmean_diff: {mean_diff:.4f}")
        wandb.log({"ood": acc if acc>0.5 else 1-acc, "mean_diff": abs(mean_diff), "importance": importance}, step=l)

layer_sweep()

Honesty Control

In [ ]:
import json

json.load()